<a href="https://colab.research.google.com/github/pedro-de-bastos/cs156/blob/master/Assignment_6_Full_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM Fitting

In [0]:

import glob
import numpy as np
from PIL import Image

#Getting the file names. Note that I already separate between male and female clothes.
#male = glob.glob("/content/drive/My Drive/CS156/Assignment 4 Data Pictures/Man's Clothing - n03746330/n03746330*")
#female = glob.glob("/content/drive/My Drive/CS156/Assignment 4 Data Pictures/Woman's Clothing - n04596852/n04596852*")
male = glob.glob("n03746330*")
female = glob.glob("n04596852*")

male_data = []
female_data = []

#Opening the image files, resizing them to standardize the procedure, and transforming the image into an array
for file in male:
    pic = Image.open(file)
    pic = pic.resize((200, 200), resample=0)
    pic = np.asarray(pic)
    male_data.append(pic)

for file in female:
    pic = Image.open(file)
    pic = pic.resize((200, 200), resample=0)
    pic = np.asarray(pic)
    female_data.append(pic)

#Flattening the arrays to make the data readable by SKLearn classifiers
for index, file in enumerate(female_data):
    female_data[index] = np.ndarray.flatten(female_data[index])

for index, file in enumerate(male_data):
    male_data[index] = np.ndarray.flatten(male_data[index])

#Separating our data for training
trainf = female_data[0:600]
trainm = male_data[0:600]
train = np.asarray(trainf + trainm)
train_classes = np.asarray(list(np.ones(len(trainf)))+list(np.zeros(len(trainm))))

#Separating our data for testing
testf = female_data[600:]
testm = male_data[600:]
test = np.asarray(testf + testm)
test_classes = np.asarray(list(np.ones(len(testf)))+list(np.zeros(len(testm))))

#Creating my model
from sklearn.svm import SVC
clf_rbf = SVC(kernel='rbf', gamma='auto')
clf_poly = SVC(kernel='poly', degree=2, gamma='auto')
clf_lin = SVC(kernel='linear', gamma='auto')

#Creating a function that will fit my model to both test and train data, and return accuracy rates
from sklearn.metrics import accuracy_score
def fit_and_accuracy(X_train, y_train, X_test, y_test, clf):
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    train_accuracy = accuracy_score(y_train_pred, y_train)
    y_test_pred = clf.predict(X_test)
    test_accuracy = accuracy_score(y_test_pred, y_test)
    print("Your accuracy score on the training data is:", train_accuracy)
    print("Your accuracy score on the test data is:", test_accuracy)
    return train_accuracy, test_accuracy

fit_and_accuracy(train, train_classes, test, test_classes, clf_poly)
fit_and_accuracy(train, train_classes, test, test_classes, clf_lin)
fit_and_accuracy(train, train_classes, test, test_classes, clf_rbf)

# VGG16

In [0]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="train",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="test", target_size=(224,224))

#The Neural Network
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=1,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

model.evaluate_generator(testdata)
model.metrics_names